<a href="https://colab.research.google.com/github/Clinton-Nyaore/Clinton-Nyaore/blob/main/Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [624]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/Data/spaceship-titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Data/spaceship-titanic/test.csv')
test_ids = test['PassengerId']

In [625]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [626]:
def clean(data):
  data = data.drop(['Name', 'Cabin', 'PassengerId'], axis=1)

  cols = ['Age','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
  for col in cols:
      data[col].fillna(data[col].median(), inplace=True)

  return data

train = clean(train)
test = clean(test)

In [627]:
train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [628]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
columns = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

for col in columns:
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

In [629]:
train.isna().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [630]:
train['Transported'] = le.fit_transform(train['Transported'])
y = train['Transported']
X = train.drop('Transported', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [631]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.decomposition import PCA
pca = PCA()

from sklearn.pipeline import Pipeline
rf = RandomForestClassifier(random_state=1, max_depth=7)
steps = [('scaler',scaler), ('pca',pca), ('model',rf)]
model = Pipeline(steps)
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
print('Train Accuracy', accuracy_score(y_train, model.predict(X_train)))
print('Test Accuracy', accuracy_score(y_test, model.predict(X_test)))

Train Accuracy 0.8175150992234685
Test Accuracy 0.7786083956296722


In [632]:
submission_preds = model.predict(test)

In [633]:
df = pd.DataFrame({'PassengerId': test_ids.values,
                   'Transported': submission_preds,
                  })

In [634]:
df['Transported'] = df['Transported'].astype('bool')

In [635]:
df.to_csv("submission.csv", index=False)